In [25]:
import warnings
warnings.filterwarnings('ignore') 

In [26]:
import pandas as pd
import numpy as np
import qgrid
from numpy import random
import scipy as sc
import numpy.matlib as matlib
import time
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix
from string import ascii_uppercase

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

db = np.loadtxt('segmentationT.data', delimiter = ',')

Y = db[:, 0]
Y = Y.astype(int)
X = db[:, 1:]



In [27]:
def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)

In [7]:
def fisher(X):
    tam = len(X)
    res= np.zeros([19,19])
    for i in range(tam):
        for j in range(tam):
            
            

SyntaxError: unexpected EOF while parsing (<ipython-input-7-59981f9291a4>, line 7)

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

In [29]:
def select_features(modelo, n_features, fwd, fltg):

    sfs = SFS(modelo, 
           k_features=n_features,
           forward=fwd,
           floating=fltg,
           verbose=1,
           scoring='accuracy',
           cv=0)
    
    return sfs

def seleccion(features):
    
    fwd = False
    fltg = False
        
    #Para calcular el costo computacional
    tiempo_i = time.time()

    accuracy_list = np.zeros([4])
    precision_list = np.zeros([4,7])
    recall_list = np.zeros([4,7])
    f_list = np.zeros([4,7])
    TN_list = np.zeros([4])
    FP_list = np.zeros([4])
    FN_list = np.zeros([4])
    TP_list = np.zeros([4])   
    #Implemetamos la metodología de validación 

    Errores = np.ones(4)
    j = 0
    model = svm.SVC(decision_function_shape='ovo', kernel='rbf', C = 10, gamma = 1)
    #model = RandomForestClassifier(n_estimators=20, max_features = 2)#Son dos porque sino no se puede ejecutar con 3 variables
    
    for j in range(4):

        Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.25) # Modificar metodología de validación
        scaler = MinMaxScaler()#Escala entre 0 y 1
        Xtrain = scaler.fit_transform(Xtrain)
        Xtest = scaler.transform(Xtest)

        #Aquí se entrena y se valida el modelo haciendo selección de características con diferentes estrategias

        #Complete el código llamando el método select_features con los parámetros correspondientes para responder el
        #Ejercicio 3.1
        

        sf = select_features(model, features, fwd, fltg)

        #Complete el códpip install mlxtend  igo para entrenar el modelo con las características seleccionadas. Tenga en cuenta
        #la metodología de validación aplicada para que pase las muestras de entrenamiento correctamente.
        sf =  sf.fit(Xtrain, Ytrain)

        X_train_sfs = sf.transform(Xtrain)
        X_test_sfs = sf.transform(Xtest)
        XN = sf.transform(X)

        # Fit the estimator using the new feature subset
        # and make a prediction on the test data
        model.fit(X_train_sfs, Ytrain)
        pred = model.predict(X_test_sfs)
        
        #code for calculating accuracy 
        _accuracy_ = accuracy_score(Ytest, pred, normalize=True)
        accuracy_list[j] = _accuracy_

        #code for calculating recall 
        _recalls_ = recall_score(Ytest, pred, average=None)
        recall_list[j] = _recalls_

        #code for calculating precision 
        _precisions_ = precision_score(Ytest, pred, average=None)
        precision_list[j] = _precisions_
        
        _f_score_ = f1_score(Ytest, pred, average=None)
        f_list[j] = _f_score_


        #code for calculating confusion matrix 
        _confusion_matrix_ = confusion_matrix(Ytest, pred)
        TN_list[j] = _confusion_matrix_[0][0]
        FP_list[j] = _confusion_matrix_[0][1]
        FN_list[j] = _confusion_matrix_[1][0]
        TP_list[j] = _confusion_matrix_[1][1]
        
        Errores[j] = classification_error(pred, Ytest)
        
    print(sf.k_feature_idx_)    
    return XN, X_train_sfs, X_test_sfs, str(np.mean(accuracy_list)), str(np.std(accuracy_list)), str(np.mean(recall_list)), str(np.std(recall_list)), str(np.mean(precision_list)), str(np.std(precision_list)),  str(np.mean(f_list)), str(np.std(f_list)), str(np.mean(Errores)), str(np.std(Errores)), str(time.time()-tiempo_i)

In [30]:
pd.options.mode.chained_assignment = None

randn = np.random.randn
df_types = pd.DataFrame({'# de características seleccionadas' : pd.Series([19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3])})
df_types["Eficiencia"] = ""
df_types["Int_Eficiencia"] = ""
df_types["Sensibilidad"] = ""
df_types["Int_Sensibilidad"] = ""
df_types["Precision"] = ""
df_types["Int_Precision"] = ""
df_types["F-Score"] = ""
df_types["Int_F-Score"] = ""
df_types["Error_Prueba"] = ""
df_types["Int_error"] = ""
df_types["Tiempo de ejecución"] = ""
df_types.set_index(['# de características seleccionadas'], inplace=True)

for k in df_types.index:
    XN, XtrainN, XtestN, Acc, IntAcc, Sen, IntSen, Pre, IntPre, f, IntF, error, stdError, tiempo = seleccion(k)
    print(k)
    df_types["Eficiencia"][k] = Acc
    df_types["Int_Eficiencia"][k] = IntAcc
    df_types["Sensibilidad"][k] = Sen
    df_types["Int_Sensibilidad"][k] = IntSen
    df_types["Precision"][k] = Pre
    df_types["Int_Precision"][k] = IntPre
    df_types["F-Score"][k] = f
    df_types["Int_F-Score"][k] = IntF
    df_types["Error_Prueba"][k] = error
    df_types["Int_error"][k] = stdError
    df_types["Tiempo de ejecución"][k] = tiempo
    
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18)
19


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.5s finished
Features: 18/18[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.5s finished
Features: 18/18[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.5s finished
Features: 18/18[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.5s finished
Features: 18/18[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18)
18


[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.5s finished
Features: 18/17[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    0.5s finished
Features: 17/17[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.5s finished
Features: 18/17[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    0.5s finished
Features: 17/17[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.6s finished
Features: 18/17[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    0.5s finished
Features: 17/17[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Par

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 17, 18)
17


...

[Parallel(n_jobs=1)]: Done  14 out of  14 | elapsed:    0.3s finished
Features: 13/4[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    0.3s finished
Features: 12/4[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.3s finished
Features: 11/4[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.3s finished
Features: 10/4[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.2s finished
Features: 9/4[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.2s finished
Features: 8/4[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_

(1, 13, 15, 17)
4


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.5s finished
Features: 18/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    0.5s finished
Features: 17/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    0.5s finished
Features: 16/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.4s finished
Features: 15/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.4s finished
Features: 14/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  14 out of  14 | elapsed:    0.3s finished
Features: 13/3[Parallel(

Features: 17/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    0.5s finished
Features: 16/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.5s finished
Features: 15/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.4s finished
Features: 14/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  14 out of  14 | elapsed:    0.4s finished
Features: 13/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    0.3s finished
Features: 12/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.3s finished
Features: 

(1, 13, 15)
3


In [31]:
qgrid_widget.get_changed_df()

,Eficiencia,Int_Eficiencia,Sensibilidad,Int_Sensibilidad,Precision,Int_Precision,F-Score,Int_F-Score,Error_Prueba,Int_error,Tiempo de ejecución
# de características seleccionadas,,,,,,,,,,,
19,0.9697231833910034,0.002869052586812623,0.9693246195289014,0.03377110853654812,0.969667171185579,0.03934786223100783,0.969320583334259,0.03449891330407207,0.03027681660899654,0.002869052586812629,0.23821568489074707
18,0.9593425605536332,0.009031407014628522,0.959938996212478,0.044801740575713435,0.9603757450673898,0.04972559630423971,0.9598451339793416,0.04438660887628973,0.04065743944636678,0.009031407014628505,2.774162530899048
17,0.9619377162629759,0.008022161328283461,0.9638607074623923,0.042120324540097426,0.9651421078941134,0.04889403839848884,0.9642792068803319,0.04351869857092088,0.03806228373702422,0.00802216132828348,5.30962872505188
16,0.9675605536332179,0.004474083232175023,0.967803296921826,0.04111089094896041,0.9679577863445872,0.04320995256122556,0.96746668761565,0.03787787620491524,0.032439446366782004,0.004474083232174999,7.425947666168213
15,0.9666955017301038,0.008057065748355173,0.9670450931317187,0.037689007170831054,0.9673343643671828,0.046069930874257585,0.9670378618239585,0.04046907100318125,0.0333044982698962,0.008057065748355173,9.312493801116943
14,0.971885813148789,0.003940498952917071,0.9703006403678235,0.03594339068148879,0.9710593233362967,0.038651395139752354,0.970463382898095,0.034657522878780385,0.028114186851211073,0.003940498952917085,11.32704758644104
13,0.9645328719723183,0.00605536332179933,0.9643908538821753,0.05289930969397777,0.9640310519957865,0.04655058879369214,0.9635219086287433,0.04401766552728176,0.03546712802768166,0.006055363321799308,13.192347526550293
12,0.9653979238754326,0.007441457843462503,0.9658982747001879,0.043710083734640505,0.9658633289758918,0.04797613420662917,0.9655503479434564,0.04278979974746364,0.03460207612456748,0.00744145784346248,14.579661130905151
11,0.9641003460207612,0.0037457846184447844,0.9631821053236934,0.0469772608205575,0.963338486400562,0.045143540840956946,0.9631218605579459,0.0448225548874298,0.035899653979238756,0.0037457846184448034,16.46354627609253


In [38]:
XN, XtrainN, XtestN, Acc, IntAcc, Sen, IntSen, Pre, IntPre, f, IntF, error, stdError, tiempo = seleccion(14)
print(error)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.5s finished
Features: 18/14[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    0.5s finished
Features: 17/14[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    0.5s finished
Features: 16/14[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.4s finished
Features: 15/14[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.4s finished
Features: 14/14[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.6s finished
Features: 18/14[Par

(0, 1, 4, 5, 7, 8, 9, 10, 11, 13, 14, 15, 17, 18)
0.03503460207612456


In [39]:
XN = X[:, [0, 1, 3, 4, 5, 7, 9, 10, 12, 13, 14, 15, 17, 18]]

In [40]:
print(XN)

[[ 1.4000000e+02  1.2500000e+02  0.0000000e+00 ... -7.8888890e+00
   5.4563490e-01 -1.1218182e+00]
 [ 1.8800000e+02  1.3300000e+02  0.0000000e+00 ... -8.3333330e+00
   5.3858024e-01 -9.2481726e-01]
 [ 1.0500000e+02  1.3900000e+02  0.0000000e+00 ... -7.6666665e+00
   5.3262790e-01 -9.6594584e-01]
 ...
 [ 1.2800000e+02  1.6100000e+02  0.0000000e+00 ... -6.4444447e+00
   5.4091770e-01 -1.9963073e+00]
 [ 1.5000000e+02  1.5800000e+02  0.0000000e+00 ... -7.0000000e+00
   5.0308645e-01 -1.9434487e+00]
 [ 1.2400000e+02  1.6200000e+02  1.1111111e-01 ... -7.4444447e+00
   4.7993130e-01 -2.0293121e+00]]


# Naive bayes

In [41]:
from sklearn.naive_bayes import GaussianNB

In [52]:
def model_NB(impresion = False):

    tiempo_i = time.time()

    accuracy_list = np.zeros([4])
    precision_list = np.zeros([4,7])
    recall_list = np.zeros([4,7])
    f_list = np.zeros([4,7])
    TN_list = np.zeros([4])
    FP_list = np.zeros([4])
    FN_list = np.zeros([4])
    TP_list = np.zeros([4])   
    errores = np.zeros(4)
    nb = GaussianNB()
    for j in range(4):
        
        
        Xtrain, Xtest, Ytrain, Ytest = train_test_split(XN, Y, test_size=0.25) # Modificar metodología de validación
        scaler = MinMaxScaler()#Escala entre 0 y 1
        Xtrain = scaler.fit_transform(Xtrain)
        Xtest = scaler.transform(Xtest)
        
        nb.fit(Xtrain, Ytrain)
        pred = nb.predict(Xtest)
        
        #code for calculating accuracy 
        _accuracy_ = accuracy_score(Ytest, pred, normalize=True)
        accuracy_list[j] = _accuracy_

        #code for calculating recall 
        _recalls_ = recall_score(Ytest, pred, average=None)
        recall_list[j] = _recalls_

        #code for calculating precision 
        _precisions_ = precision_score(Ytest, pred, average=None)
        precision_list[j] = _precisions_
        
        _f_score_ = f1_score(Ytest, pred, average=None)
        f_list[j] = _f_score_


        #code for calculating confusion matrix 
        _confusion_matrix_ = confusion_matrix(Ytest, pred)
        TN_list[j] = _confusion_matrix_[0][0]
        FP_list[j] = _confusion_matrix_[0][1]
        FN_list[j] = _confusion_matrix_[1][0]
        TP_list[j] = _confusion_matrix_[1][1]
                
        errores[j] = classification_error(pred, Ytest)
          
    if impresion == True:
        cm = confusion_matrix(Ytest, pred)
        columnas = ['Clase %s'%(i) for i in list(ascii_uppercase)[0:len(np.unique(pred))]]
        df_cm = pd.DataFrame(cm,index = columnas, columns = columnas)
        
        grafica = sns.heatmap(df_cm, cmap = 'Blues', annot = True)
        
        grafica.set(xlabel = 'Verdaderos', ylabel = 'Predicciones')
#         print(classification_report(Ytest, pred))

    return str(np.mean(accuracy_list)), str(np.std(accuracy_list)), str(np.mean(recall_list)), str(np.std(recall_list)), str(np.mean(precision_list)), str(np.std(precision_list)),  str(np.mean(f_list)), str(np.std(f_list)), str(np.mean(errores)), str(np.std(errores)), str(time.time()-tiempo_i)

In [53]:
Acc, IntAcc, Sen, IntSen, Pre, IntPre, f, IntF, error, stdError, tiempo = model_NB(impresion = False)
print('Eficiencia',Acc, ' Int_Eficiencia', IntAcc,' Sensibilidad', Sen, ' Int_Sensibilidad',IntSen,' Precision', Pre, ' Int_Precision',IntPre,' F-Score', f, ' Int_F-Score',IntF,' Error_Prueba', error,' Int_Error', stdError,' Tiempo ejecución', tiempo)

Eficiencia 0.8486159169550173  Int_Eficiencia 0.01214158204811262  Sensibilidad 0.8450434379792536  Int_Sensibilidad 0.2071615181473435  Precision 0.8607612384817419  Int_Precision 0.15317761552142636  F-Score 0.8388589016932934  Int_F-Score 0.18289435330691303  Error_Prueba 0.1513840830449827  Int_Error 0.012141582048112632  Tiempo ejecución 0.020018815994262695


# KNN

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


In [49]:
def model_knn(k, impresion = False):

    tiempo_i = time.time()
    accuracy_list = np.zeros([4])
    precision_list = np.zeros([4,7])
    recall_list = np.zeros([4,7])
    f_list = np.zeros([4,7])
    TN_list = np.zeros([4])
    FP_list = np.zeros([4])
    FN_list = np.zeros([4])
    TP_list = np.zeros([4])   
    errores = np.zeros(4)
    erroresMAPE = np.zeros(4)
    knn = KNeighborsClassifier(n_neighbors = k)
    for j in range(4):
        Xtrain, Xtest, Ytrain, Ytest = train_test_split(XN, Y, test_size=0.25) # Modificar metodología de validación
        scaler = MinMaxScaler()#Escala entre 0 y 1
        Xtrain = scaler.fit_transform(Xtrain)
        Xtest = scaler.transform(Xtest)
        
        knn.fit(Xtrain, Ytrain)
        pred = knn.predict(Xtest)
        
        #code for calculating accuracy 
        _accuracy_ = accuracy_score(Ytest, pred, normalize=True)
        accuracy_list[j] = _accuracy_

        #code for calculating recall 
        _recalls_ = recall_score(Ytest, pred, average=None)
        recall_list[j] = _recalls_

        #code for calculating precision 
        _precisions_ = precision_score(Ytest, pred, average=None)
        precision_list[j] = _precisions_
        
        _f_score_ = f1_score(Ytest, pred, average=None)
        f_list[j] = _f_score_


        #code for calculating confusion matrix 
        _confusion_matrix_ = confusion_matrix(Ytest, pred)
        TN_list[j] = _confusion_matrix_[0][0]
        FP_list[j] = _confusion_matrix_[0][1]
        FN_list[j] = _confusion_matrix_[1][0]
        TP_list[j] = _confusion_matrix_[1][1]
                
        errores[j] = classification_error(pred, Ytest)
           
    if impresion == True:
        cm = confusion_matrix(Ytest, pred)
        columnas = ['Clase %s'%(i) for i in list(ascii_uppercase)[0:len(np.unique(pred))]]
        df_cm = pd.DataFrame(cm,index = columnas, columns = columnas)
        
        grafica = sns.heatmap(df_cm, cmap = 'Blues', annot = True)
        
        grafica.set(xlabel = 'Verdaderos', ylabel = 'Predicciones')
#         print(classification_report(Ytest, pred))

    return str(np.mean(accuracy_list)), str(np.std(accuracy_list)), str(np.mean(recall_list)), str(np.std(recall_list)), str(np.mean(precision_list)), str(np.std(precision_list)),  str(np.mean(f_list)), str(np.std(f_list)), str(np.mean(errores)), str(np.std(errores)), str(time.time()-tiempo_i)


In [50]:
pd.options.mode.chained_assignment = None

randn = np.random.randn
df_types = pd.DataFrame({
    'Numero de vecinos' : pd.Series(['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'])})
df_types["Eficiencia"] = ""
df_types["Int_Eficiencia"] = ""
df_types["Sensibilidad"] = ""
df_types["Int_Sensibilidad"] = ""
df_types["Precision"] = ""
df_types["Int_Precision"] = ""
df_types["F-Score"] = ""
df_types["Int_F-Score"] = ""
df_types["Error_Prueba"] = ""
df_types["Int_error"] = ""
df_types["Tiempo de ejecución"] = ""
df_types.set_index(['Numero de vecinos'], inplace=True)

i = 0
for k in df_types.index:
    Acc, IntAcc, Sen, IntSen, Pre, IntPre, f, IntF, error, stdError, tiempo = model_knn(int(k), impresion = False)
    df_types["Eficiencia"][i] = Acc
    df_types["Int_Eficiencia"][i] = IntAcc
    df_types["Sensibilidad"][i] = Sen
    df_types["Int_Sensibilidad"][i] = IntSen
    df_types["Precision"][i] = Pre
    df_types["Int_Precision"][i] = IntPre
    df_types["F-Score"][i] = f
    df_types["Int_F-Score"][i] = IntF
    df_types["Error_Prueba"][i] = error
    df_types["Int_error"][i] = stdError
    df_types["Tiempo de ejecución"][i] = tiempo
    
    i += 1
    
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [51]:
qgrid_widget.get_changed_df()

,Eficiencia,Int_Eficiencia,Sensibilidad,Int_Sensibilidad,Precision,Int_Precision,F-Score,Int_F-Score,Error_Prueba,Int_error,Tiempo de ejecución
Numero de vecinos,,,,,,,,,,,
1,0.9675605536332179,0.006162546214868086,0.9659595738307892,0.049277906209545294,0.9671493284900736,0.03242179362002127,0.9663030273096362,0.03913104669008897,0.032439446366782004,0.006162546214868082,0.08407759666442871
2,0.9550173010380623,0.004410916534249805,0.9535099437866323,0.06747079770956385,0.9537977558601295,0.044055032363385864,0.9526057532800575,0.04961378777698937,0.04498269896193771,0.004410916534249814,0.09007763862609863
3,0.9610726643598615,0.003118988992615917,0.9593892634417174,0.0517649991227288,0.9609843690598293,0.04349555764046978,0.9594752162270976,0.04111866671238931,0.03892733564013841,0.0031189889926159053,0.09001541137695312
4,0.9480968858131488,0.002735534308104141,0.9474411169262845,0.06629409817560585,0.9479087260391165,0.05324279060505198,0.947025522181657,0.055849491523488115,0.051903114186851215,0.0027355343081041355,0.09301614761352539
5,0.9502595155709341,0.007578466897895854,0.9489364928245448,0.06433529162017411,0.9504162477678865,0.04741569535017161,0.9488160702815965,0.04980131351452486,0.049740484429065736,0.007578466897895861,0.0960693359375
6,0.9515570934256055,0.009316894129990469,0.9542156843340664,0.05464624494988493,0.9526582658864722,0.046309508629560445,0.9524375332732163,0.0413106151101128,0.048442906574394456,0.00931689412999049,0.09501147270202637
7,0.9515570934256055,0.005332538064852047,0.9513969627583458,0.05898650153231492,0.9522866573280119,0.045378598999363196,0.950999899274443,0.045303806373288555,0.04844290657439446,0.005332538064852056,0.09694337844848633
8,0.9437716262975779,0.003566700368181368,0.9439426954873485,0.06247767151057361,0.943428564121956,0.05125684984302385,0.9427211337720819,0.049548990131891964,0.056228373702422146,0.003566700368181367,0.10001301765441895
9,0.9511245674740485,0.009505735716191985,0.9520249946518614,0.0564049183185105,0.9516870558622356,0.043937757031143145,0.9512414037062397,0.04495027796770767,0.048875432525951557,0.009505735716192001,0.09992742538452148


# Redes Neuronales Artificiales

In [54]:
from sklearn.neural_network import MLPClassifier

In [55]:
def model_MLP(nCO, nPC, impresion = False):
    if(nCO==1):
        mlp=MLPClassifier(activation='tanh',max_iter = 1000,hidden_layer_sizes=(nPC))
    else:
        mlp=MLPClassifier(activation='tanh',max_iter = 1000,hidden_layer_sizes=(nPC,nPC))
        
    tiempo_i = time.time()

    accuracy_list = np.zeros([4])
    precision_list = np.zeros([4,7])
    recall_list = np.zeros([4,7])
    f_list = np.zeros([4,7])
    TN_list = np.zeros([4])
    FP_list = np.zeros([4])
    FN_list = np.zeros([4])
    TP_list = np.zeros([4])   
    errores = np.zeros(4)
    erroresMAPE = np.zeros(4)
    for j in range(4):
        Xtrain, Xtest, Ytrain, Ytest = train_test_split(XN, Y, test_size=0.25) # Modificar metodología de validación
        scaler = MinMaxScaler()#Escala entre 0 y 1
        Xtrain = scaler.fit_transform(Xtrain)
        Xtest = scaler.transform(Xtest)
        
        mlp.fit(Xtrain, Ytrain)
        pred = mlp.predict(Xtest)
        
        #code for calculating accuracy 
        _accuracy_ = accuracy_score(Ytest, pred, normalize=True)
        accuracy_list[j] = _accuracy_

        #code for calculating recall 
        _recalls_ = recall_score(Ytest, pred, average=None)
        recall_list[j] = _recalls_

        #code for calculating precision 
        _precisions_ = precision_score(Ytest, pred, average=None)
        precision_list[j] = _precisions_
        
        _f_score_ = f1_score(Ytest, pred, average=None)
        f_list[j] = _f_score_


        #code for calculating confusion matrix 
        _confusion_matrix_ = confusion_matrix(Ytest, pred)
        TN_list[j] = _confusion_matrix_[0][0]
        FP_list[j] = _confusion_matrix_[0][1]
        FN_list[j] = _confusion_matrix_[1][0]
        TP_list[j] = _confusion_matrix_[1][1]
                
        
        errores[j] = classification_error(pred, Ytest)
       
    
    if impresion == True:
        cm = confusion_matrix(Ytest, pred)
        columnas = ['Clase %s'%(i) for i in list(ascii_uppercase)[0:len(np.unique(pred))]]
        df_cm = pd.DataFrame(cm,index = columnas, columns = columnas)
        
        grafica = sns.heatmap(df_cm, cmap = 'Blues', annot = True)
        
        grafica.set(xlabel = 'Verdaderos', ylabel = 'Predicciones')
#         print(classification_report(Ytest, pred))

    return str(np.mean(accuracy_list)), str(np.std(accuracy_list)), str(np.mean(recall_list)), str(np.std(recall_list)), str(np.mean(precision_list)), str(np.std(precision_list)),  str(np.mean(f_list)), str(np.std(f_list)), str(np.mean(errores)), str(np.std(errores)), str(time.time()-tiempo_i)
    

In [56]:
pd.options.mode.chained_assignment = None

randn = np.random.randn
df_types = pd.DataFrame({
    'N. de capas ocultas' : pd.Series([1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2]),
    'Neuronas por capa' : pd.Series([5,10,15,20,25,30,35,40,45,50,5,10,15,20,25,30,35,40,45,50])})
df_types["Eficiencia"] = ""
df_types["Int_Eficiencia"] = ""
df_types["Sensibilidad"] = ""
df_types["Int_Sensibilidad"] = ""
df_types["Precision"] = ""
df_types["Int_Precision"] = ""
df_types["F-Score"] = ""
df_types["Int_F-Score"] = ""
df_types["Error_Prueba"] = ""
df_types["Int_error"] = ""
df_types["Tiempo de ejecución"] = ""
df_types.set_index(['N. de capas ocultas','Neuronas por capa'], inplace=True)
    
for n, k in df_types.index:
    Acc, IntAcc, Sen, IntSen, Pre, IntPre, f, IntF, error, stdError, tiempo = model_MLP(n, k, impresion = False)
    df_types["Eficiencia"][n,k] = Acc
    df_types["Int_Eficiencia"][n,k] = IntAcc
    df_types["Sensibilidad"][n,k] = Sen
    df_types["Int_Sensibilidad"][n,k] = IntSen
    df_types["Precision"][n,k] = Pre
    df_types["Int_Precision"][n,k] = IntPre
    df_types["F-Score"][n,k] = f
    df_types["Int_F-Score"][n,k] = IntF
    df_types["Error_Prueba"][n,k] = error
    df_types["Int_error"][n,k] = stdError
    df_types["Tiempo de ejecución"][n,k] = tiempo
    
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)

In [57]:
qgrid_widget.get_changed_df()

Eficiencia  \
N. de capas ocultas Neuronas por capa                       
1                   5                  0.9243079584775087   
                    10                 0.9571799307958477   
                    15                 0.9571799307958477   
                    20                 0.9597750865051904   
                    25                 0.9593425605536332   
                    30                 0.9684256055363321   
                    35                 0.9649653979238755   
                    40                 0.9636678200692042   
                    45                 0.9602076124567474   
                    50                 0.9602076124567474   
2                   5                   0.948961937716263   
                    10                 0.9658304498269896   
                    15                 0.9658304498269896   
                    20                 0.9697231833910034   
                    25                 0.9727508650519032   
                    30                 0.9606401384083045   
                    35                  0.972750865051903   
                    40                 0.9645328719723183   
                    45                  0.972318339100346   
                    50                 0.9658304498269896   

                                              Int_Eficiencia  \
N. de capas ocultas Neuronas por capa                          
1                   5                   0.005786802837482548   
                    10                  0.004638324089430607   
                    15                  0.003088853126532368   
                    20                  0.006742412313200631   
                    25                  0.007590799643072792   
                    30                  0.005656010739888421   
                    35                  0.009815575880545165   
                    40                  0.007441457843462458   
                    45                  0.004237871527306556   
                    50                  0.007441457843462471   
2                   5                    0.00919562786568741   
                    10                 0.0037457846184448264   
                    15                  0.009266559379597148   
                    20                  0.006177706253064766   
                    25                  0.004474083232174993   
                    30                  0.006282802355680784   
                    35                  0.007963647335433376   
                    40                  0.005802944578286666   
                    45                 0.0017301038062284002   
                    50                  0.004303578880218969   

                                             Sensibilidad  \
N. de capas ocultas Neuronas por capa                       
1                   5                  0.9244231177351362   
                    10                 0.9561732015907463   
                    15                 0.9573156055559807   
                    20                 0.9602595830902992   
                    25                 0.9588508766601903   
                    30                 0.9680481027758744   
                    35                 0.9654957632215334   
                    40                 0.9640680262671035   
                    45                 0.9612854589464062   
                    50                 0.9600595649850465   
2                   5                  0.9505338771366246   
                    10                 0.9666284776178029   
                    15                 0.9649885917257348   
                    20                 0.9684581308535432   
                    25                 0.9727288168763257   
                    30                 0.9608531720919108   
                    35                 0.9727130806043984   
                    40                 0.9640614351822235   
                    45                  0.973221305830334   
              

# Random forest

In [68]:
from sklearn.ensemble import RandomForestClassifier # Se llama a la librería del método Random Forest

In [59]:
def model_RF(estimadores, caracteristicas, impresion = False):
        
    tiempo_i = time.time()
    
    accuracy_list = np.zeros([4])
    precision_list = np.zeros([4,7])
    recall_list = np.zeros([4,7])
    f_list = np.zeros([4,7])
    TN_list = np.zeros([4])
    FP_list = np.zeros([4])
    FN_list = np.zeros([4])
    TP_list = np.zeros([4])   
    errores = np.zeros(4)
    erroresMAPE = np.zeros(4)
    
    RF = RandomForestClassifier(n_estimators=estimadores, max_features =caracteristicas)
    
    for j in range(4):
        Xtrain, Xtest, Ytrain, Ytest = train_test_split(XN, Y, test_size=0.25) # Modificar metodología de validación
        scaler = MinMaxScaler()#Escala entre 0 y 1
        Xtrain = scaler.fit_transform(Xtrain)
        Xtest = scaler.transform(Xtest)
        
        RF.fit(Xtrain, Ytrain)
        pred = RF.predict(Xtest)
        
        #code for calculating accuracy 
        _accuracy_ = accuracy_score(Ytest, pred, normalize=True)
        accuracy_list[j] = _accuracy_

        #code for calculating recall 
        _recalls_ = recall_score(Ytest, pred, average=None)
        recall_list[j] = _recalls_

        #code for calculating precision 
        _precisions_ = precision_score(Ytest, pred, average=None)
        precision_list[j] = _precisions_
        
        _f_score_ = f1_score(Ytest, pred, average=None)
        f_list[j] = _f_score_

        #code for calculating confusion matrix 
        _confusion_matrix_ = confusion_matrix(Ytest, pred)
        TN_list[j] = _confusion_matrix_[0][0]
        FP_list[j] = _confusion_matrix_[0][1]
        FN_list[j] = _confusion_matrix_[1][0]
        TP_list[j] = _confusion_matrix_[1][1]
                
        
        errores[j] = classification_error(pred, Ytest)
       
    
    if impresion == True:
        cm = confusion_matrix(Ytest, pred)
        columnas = ['Clase %s'%(i) for i in list(ascii_uppercase)[0:len(np.unique(pred))]]
        df_cm = pd.DataFrame(cm,index = columnas, columns = columnas)
        
        grafica = sns.heatmap(df_cm, cmap = 'Blues', annot = True)
        
        grafica.set(xlabel = 'Verdaderos', ylabel = 'Predicciones')
#         print(classification_report(Ytest, pred))

    return str(np.mean(accuracy_list)), str(np.std(accuracy_list)), str(np.mean(recall_list)), str(np.std(recall_list)), str(np.mean(precision_list)), str(np.std(precision_list)),  str(np.mean(f_list)), str(np.std(f_list)), str(np.mean(errores)), str(np.std(errores)), str(time.time()-tiempo_i)
    

In [60]:
pd.options.mode.chained_assignment = None

randn = np.random.randn
df_types = pd.DataFrame({
    'Numero de arboles' : pd.Series([5,5,5,5,10,10,10,10,20,20,20,20,50,50,50,50,100,100,100,100]), 
    'Variables analizadas por nodo' : pd.Series([2,5,10,13,2,5,10,13,2,5,10,13,2,5,10,13,2,5,10,13])})
df_types["Eficiencia"] = ""
df_types["Int_Eficiencia"] = ""
df_types["Sensibilidad"] = ""
df_types["Int_Sensibilidad"] = ""
df_types["Precision"] = ""
df_types["Int_Precision"] = ""
df_types["F-Score"] = ""
df_types["Int_F-Score"] = ""
df_types["Error_Prueba"] = ""
df_types["Int_error"] = ""
df_types["Tiempo de ejecución"] = ""
df_types.set_index(['Numero de arboles','Variables analizadas por nodo'], inplace=True)

for n, k in df_types.index:
    Acc, IntAcc, Sen, IntSen, Pre, IntPre, f, IntF, error, stdError, tiempo = model_RF(n, k, impresion = False)
    df_types["Eficiencia"][n,k] = Acc
    df_types["Int_Eficiencia"][n,k] = IntAcc
    df_types["Sensibilidad"][n,k] = Sen
    df_types["Int_Sensibilidad"][n,k] = IntSen
    df_types["Precision"][n,k] = Pre
    df_types["Int_Precision"][n,k] = IntPre
    df_types["F-Score"][n,k] = f
    df_types["Int_F-Score"][n,k] = IntF
    df_types["Error_Prueba"][n,k] = error
    df_types["Int_error"][n,k] = stdError
    df_types["Tiempo de ejecución"][n,k] = tiempo


#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)

In [61]:
qgrid_widget.get_changed_df()

Eficiencia  \
Numero de arboles Variables analizadas por nodo                       
5                 2                              0.9584775086505191   
                  5                              0.9701557093425606   
                  10                             0.9744809688581315   
                  13                             0.9753460207612457   
10                2                              0.9710207612456747   
                  5                              0.9731833910034602   
                  10                             0.9740484429065743   
                  13                             0.9705882352941176   
20                2                              0.9775086505190311   
                  5                              0.9848615916955018   
                  10                              0.972318339100346   
                  13                             0.9731833910034602   
50                2                              0.9779411764705883   
                  5                              0.9775086505190311   
                  10                             0.9757785467128028   
                  13                             0.9779411764705883   
100               2                              0.9826989619377162   
                  5                              0.9779411764705881   
                  10                             0.9779411764705882   
                  13                             0.9809688581314879   

                                                        Int_Eficiencia  \
Numero de arboles Variables analizadas por nodo                          
5                 2                              0.0044109165342497945   
                  5                                0.00651666054209314   
                  10                             0.0060398962127028305   
                  13                              0.004126034608204794   
10                2                              0.0022474707710669166   
                  5                               0.008693664032111489   
                  10                              0.003670104400622236   
                  13                              0.008206602924312412   
20                2                               0.007133400736362736   
                  5                               0.005384904670410363   
                  10                              0.008650519031141885   
                  13                              0.005539034807467856   
50                2                               0.007773443233395922   
                  5                              0.0044109165342497945   
                  10                             0.0038686297188577773   
                  13                              0.003322294873645596   
100               2                               0.002996627694755841   
                  5                               0.009266559379597122   
                  10                               0.00833103818523276   
                  13                              0.002996627694755841   

                                                       Sensibilidad  \
Numero de arboles Variables analizadas por nodo                       
5                 2                              0.9592111004514823   
                  5                              0.9704119611411396   
                  10                             0.9746720931127166   
                  13                             0.9751229144559886   
10                2                              0.9718234010043206   
                  5                              0.9730177604578462   
                  10                             0.9738470895499015   
                  13                             0.9708296306584395   
20                2                              0.9778360430051184   
                  5                              0.9849290168479056   
             

# Maquinas de Soporte Vectorial con kernel lineal y con kernel RBF.

In [62]:
from sklearn.svm import SVC

In [63]:
def model_SVC(ker, C, gam, impresion = False):
    
    if gam == 0:
        gam = 'auto'
    
    tiempo_i = time.time()

    accuracy_list = np.zeros([4])
    precision_list = np.zeros([4,7])
    recall_list = np.zeros([4,7])
    f_list = np.zeros([4,7])
    TN_list = np.zeros([4])
    FP_list = np.zeros([4])
    FN_list = np.zeros([4])
    TP_list = np.zeros([4])   
    errores = np.zeros(4)
    erroresMAPE = np.zeros(4)
    PorcentajeVS = np.zeros(4)
    
    svc = SVC(gamma=gam, C=float(C),  kernel=ker, decision_function_shape='ovo' , probability = True)
    
    for j in range(4):
        Xtrain, Xtest, Ytrain, Ytest = train_test_split(XN, Y, test_size=0.25) # Modificar metodología de validación
        scaler = MinMaxScaler()#Escala entre 0 y 1
        Xtrain = scaler.fit_transform(Xtrain)
        Xtest = scaler.transform(Xtest)
        
        svc.fit(Xtrain, Ytrain)
        pred = svc.predict(Xtest)
        
        #code for calculating accuracy 
        _accuracy_ = accuracy_score(Ytest, pred, normalize=True)
        accuracy_list[j] = _accuracy_

        #code for calculating recall 
        _recalls_ = recall_score(Ytest, pred, average=None)
        recall_list[j] = _recalls_

        #code for calculating precision 
        _precisions_ = precision_score(Ytest, pred, average=None)
        precision_list[j] = _precisions_
        
        _f_score_ = f1_score(Ytest, pred, average=None)
        f_list[j] = _f_score_

        #code for calculating confusion matrix 
        _confusion_matrix_ = confusion_matrix(Ytest, pred)
        TN_list[j] = _confusion_matrix_[0][0]
        FP_list[j] = _confusion_matrix_[0][1]
        FN_list[j] = _confusion_matrix_[1][0]
        TP_list[j] = _confusion_matrix_[1][1]
                
        errores[j] = classification_error(pred, Ytest)
        PorcentajeVS[j] = svc.support_vectors_.shape[0] / Xtrain.shape[0]
    
    if impresion == True:
        cm = confusion_matrix(Ytest, pred)
        columnas = ['Clase %s'%(i) for i in list(ascii_uppercase)[0:len(np.unique(pred))]]
        df_cm = pd.DataFrame(cm,index = columnas, columns = columnas)
        
        grafica = sns.heatmap(df_cm, cmap = 'Blues', annot = True)
        
        grafica.set(xlabel = 'Verdaderos', ylabel = 'Predicciones')
#         print(classification_report(Ytest, pred))

    return str(np.mean(accuracy_list)), str(np.std(accuracy_list)), str(np.mean(recall_list)), str(np.std(recall_list)), str(np.mean(precision_list)), str(np.std(precision_list)),  str(np.mean(f_list)), str(np.std(f_list)), str(np.mean(errores)), str(np.std(errores)), str(np.mean(PorcentajeVS)), str(time.time()-tiempo_i)
    

In [64]:
pd.options.mode.chained_assignment = None

randn = np.random.randn
df_types = pd.DataFrame({
    'Kernel' : pd.Series(['linear','linear','linear','linear','linear','linear','linear','linear','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']),
    'C' : pd.Series([0.001,0.01,0.1,1,10,100,200,300,0.001,0.001,0.001,0.001,0.01,0.01,0.01,0.01,0.1,0.1,0.1,0.1,1,1,1,1,10,10,10,10,100,100,100,100,200,200,200,200,300,300,300,300]),
    'gamma' : pd.Series([0,0,0,0,0,0,0,0,0.001,0.01,0.1,1,0.001,0.01,0.1,1,0.001,0.01,0.1,1,0.001,0.01,0.1,1,0.001,0.01,0.1,1,0.001,0.01,0.1,1,0.001,0.01,0.1,1,0.001,0.01,0.1,1])})
df_types["Eficiencia"] = ""
df_types["Int_Eficiencia"] = ""
df_types["Sensibilidad"] = ""
df_types["Int_Sensibilidad"] = ""
df_types["Precision"] = ""
df_types["Int_Precision"] = ""
df_types["F-Score"] = ""
df_types["Int_F-Score"] = ""
df_types["Error_Prueba"] = ""
df_types["Int_error"] = ""
df_types["%Vectores de soporte"] = ""
df_types["Tiempo de ejecución"] = ""
df_types.set_index(['Kernel','C','gamma'], inplace=True)

for n, k, o in df_types.index:
    Acc, IntAcc, Sen, IntSen, Pre, IntPre, f, IntF, error, stdError,vs, tiempo = model_SVC(n, k, o, impresion = False)
    df_types["Eficiencia"][n,k,o] = Acc
    df_types["Int_Eficiencia"][n,k,o] = IntAcc
    df_types["Sensibilidad"][n,k,o] = Sen
    df_types["Int_Sensibilidad"][n,k,o] = IntSen
    df_types["Precision"][n,k,o] = Pre
    df_types["Int_Precision"][n,k,o] = IntPre
    df_types["F-Score"][n,k,o] = f
    df_types["Int_F-Score"][n,k,o] = IntF
    df_types["Error_Prueba"][n,k,o] = error
    df_types["Int_error"][n,k,o] = stdError
    df_types["%Vectores de soporte"][n,k,o] = vs
    df_types["Tiempo de ejecución"][n,k,o] = tiempo


#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)

In [65]:
qgrid_widget.get_changed_df()

Eficiencia         Int_Eficiencia  \
Kernel C       gamma                                               
linear 0.001   0.000  0.12326989619377163   0.007378340012643585   
       0.010   0.000   0.5609861591695502    0.03917405475583729   
       0.100   0.000                0.875  0.0037457846184447844   
       1.000   0.000   0.9269031141868513   0.006630497282333856   
       10.000  0.000   0.9368512110726643  0.0025951557093425726   
       100.000 0.000   0.9567474048442908   0.004410916534249805   
       200.000 0.000    0.958910034602076    0.00447408323217502   
       300.000 0.000   0.9558823529411765   0.007185660781071019   
rbf    0.001   0.001  0.16349480968858132   0.057491794630841725   
               0.010  0.12716262975778547   0.003566700368181365   
               0.100  0.15614186851211073    0.06288606056437972   
               1.000  0.16262975778546712    0.06393195922803419   
       0.010   0.001  0.11937716262975777    0.00955481056850109   
               0.010  0.12629757785467127  0.0038686297188577647   
               0.100  0.16219723183391002   0.047006239066498715   
               1.000   0.5705017301038062   0.061490223457456486   
       0.100   0.001  0.19377162629757783    0.06067708621712865   
               0.010  0.25346020761245674    0.02026881403782003   
               0.100                 0.75       0.04474083232175   
               1.000   0.8966262975778547   0.006742412313200634   
       1.000   0.001   0.1859861591695502    0.05712618332560228   
               0.010   0.8140138408304499   0.034999875993328565   
               0.100    0.902249134948097   0.012079792425405632   
               1.000   0.9381487889273357   0.008240726160578575   
       10.000  0.001   0.7374567474048442   0.022603360952342423   
               0.010   0.9013840830449827   0.005044076033603193   
               0.100   0.9294982698961938   0.008240726160578584   
               1.000   0.9662629757785468  0.0025951557093425543   
       100.000 0.001   0.8905709342560553   0.010041684053844204   
               0.010   0.9303633217993079   0.010621132489453868   
               0.100   0.9584775086505191   0.008206602924312406   
               1.000    0.967128027681661   0.007027714882903065   
       200.000 0.001   0.9290657439446367     0.0088218330684996   
               0.010   0.9411764705882353   0.004738084407484118   
               0.100   0.9615051903114187   0.008240726160578603   
               1.000   0.9710207612456748   0.002558858037672841   
       300.000 0.001   0.9182525951557095   0.011103371672139623   
               0.010   0.9372837370242214   0.008682897880572952   
               0.100   0.9636678200692042   0.004237871527306556   
               1.000   0.9727508650519032   0.002836262337500862   

                             Sensibilidad      Int_Sensibilidad  \
Kernel C       gamma                                              
linear 0.001   0.000  0.14285714285714285    0.3499271061118826   
       0.010   0.000   0.5806840771251854    0.4505909459307296   
       0.100   0.000     0.87552493526727   0.11539735070631052   
       1.000   0.000   0.9245032965446047   0.09297545265777077   
       10.000  0.000   0.9362675622963701    0.0748439301114078   
       100.000 0.000   0.9566022825220585  0.052693055588178724   
       200.000 0.000   0.9582201166489909     0.046434852667021   
       300.000 0.000   0.9547912009994249   0.04908451204475708   
rbf    0.001   0.001  0.17857142857142858    0.3829930462415574   
               0.010  0.14285714285714285    0.3499271061118826   
               0.100  0.17810760667903525    0.3820045665835007   
               1.000  0.17857142857142858    0.3829930462415574   
       0.010   0.001  0.14285714285714285    0.3499271061118826   
               0.010  0.14285714285714285    0.3499271061118826   
               0.100  0.18412583888774364      0.36491692410615   
               1.000   0.5866821410

In [66]:
Acc, IntAcc, Sen, IntSen, Pre, IntPre, f, IntF, error, stdError,vs, tiempo = model_SVC('rbf', 100, 0.001, impresion = False)

In [67]:
print(Acc, IntAcc, Sen, IntSen, Pre, IntPre, f, IntF, error, stdError,vs, tiempo)

0.8957612456747405 0.009346964872393712 0.8963961597808711 0.12471293980083915 0.8945930855847275 0.11449735018348477 0.8945845947150757 0.11734463014354372 0.1042387543252595 0.009346964872393731 0.608256351039261 0.7620639801025391
